In [214]:
import pandas as pd
from neo4j import GraphDatabase

In [215]:
# Database Connection

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()

In [216]:
# Helper Functions

def write_query(query, params={}):
    with driver.session(database="neo4j") as session:
        session.execute_write(lambda tx: tx.run(query, params))

def read_query(query, params={}):
    with driver.session(database="neo4j") as session:
        result = session.run(query, params)
        return [list(record.values()) for record in result]

In [217]:
# Data Mergeing

dataset_2016_targets = pd.read_csv('./data/2016_-_Cities_Emissions_Reduction_Targets_20240207.csv')
dataset_2016_ghg = pd.read_csv('./data/2016_-_Citywide_GHG_Emissions_20240207.csv')
dataset_2017_community = pd.read_csv('./data/2017_-_Cities_Community_Wide_Emissions.csv')
dataset_2017_targets = pd.read_csv('./data/2017_-_Cities_Emissions_Reduction_Targets_20240207.csv')
dataset_2023_risk = pd.read_csv('./data/2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv')

def prepare_dataset(dataset, year, col_mappings):
    dataset = dataset.rename(columns=col_mappings)
    dataset['year'] = year
    return dataset

col_mappings_2016_targets = {
    'City Name': 'city',
    'Country': 'country',
    'Baseline emissions (metric tonnes CO2e)': 'emission',
    'Target date': 'emission_target',
    'Baseline year':'baseline_year',
    'Percentage reduction target': 'target',
    'Sector':'sector'
}
col_mappings_2016_ghg = {
    'City Name': 'city',
    'Country': 'country',
    'Total CO2 emissions (metric tonnes CO2e)': 'emission',
    'City GDP': 'city_gdp',
    'Current Population': 'city_population',
    'Increase/Decrease from last year':'year_status',
    'Reporting Year':'year'
}


col_mappings_2017_community = {
    'City': 'city',
    'Country': 'country',
    'Total emissions (metric tonnes CO2e)': 'emission',
    'GDP': 'city_gdp',
    'Population': 'city_population',
    'Increase/Decrease from last year':'year_status',
    'Reporting Year':'year'
}

col_mappings_2017_targets = {
    'City': 'city',
    'Country': 'country',
    'Baseline emissions (metric tonnes CO2e)': 'emission',
    'Target date': 'emission_target',
    'Baseline year':'baseline_year',
    'Percentage reduction target': 'target',
    'Sector':'sector'
}

col_mappings_2023_risk = {
    'City': 'city',
    'Country/Area': 'country',
    'Year of publication or approval': 'year',
    'Factors considered in assessment': 'factors',
    'Population': 'city_population',
}

dataset_2016_targets_prepared = prepare_dataset(dataset_2016_targets, 2016, col_mappings_2016_targets)
dataset_2016_ghg_prepared = prepare_dataset(dataset_2016_ghg, 2016, col_mappings_2016_ghg)
dataset_2017_community_prepared = prepare_dataset(dataset_2017_community, 2017, col_mappings_2017_community)
dataset_2017_targets_prepared = prepare_dataset(dataset_2017_targets, 2017, col_mappings_2017_targets)
dataset_2023_risk_prepared = prepare_dataset(dataset_2023_risk, 2023, col_mappings_2023_risk)

merged_dataset = pd.concat([
    dataset_2016_targets_prepared,
    dataset_2016_ghg_prepared,
    dataset_2017_community_prepared,
    dataset_2017_targets_prepared,
    dataset_2023_risk_prepared
])

merged_dataset = merged_dataset.drop_duplicates(subset=['city', 'year'], keep='last')

final_selected_columns_dataset = merged_dataset[[
    'city', 'city_gdp', 'city_population', 'country', 'emission', 'emission_target', 'target', 'baseline_year', 'year', 'year_status', 'factors', 'sector'
]]

final_selected_columns_dataset = final_selected_columns_dataset.fillna(value="None")

In [218]:
# Data Ingestion

create_city_query = """
MERGE (city:City {name: $city, year: $year})
ON CREATE SET
  city.population = $population,
  city.gdp = $gdp,
  city.emission = $emission
MERGE (country:Country {name: $country})
MERGE (city)-[:LOCATED_IN]->(country)
"""

for index, row in final_selected_columns_dataset.iterrows():
    write_query(create_city_query, params={
        'city': row['city'],
        'year': row['year'],
        'population': row['city_population'],
        'gdp': row['city_gdp'],
        'emission': row['emission'],
        'country': row['country']
    })

In [219]:
# Querying the Database

query = """
MATCH (city:City)
RETURN city.name AS Name, city.population AS Population, city.gdp AS GDP
LIMIT 5
"""

response = read_query(query)
print(response)

[['Ville de Montreal', 1886481.0, 120118000000.0], ['Greater Manchester', 2762000.0, 87773000000.0], ['City of Reno', 236995.0, 10000000000.0], ['City of Minneapolis', 407207.0, 235700000000.0], ['City of London, ON', 375000.0, 15400000000.0]]


In [220]:
# Close the connection

driver.close()